# CARTOLA FC - 2020

Cartola FC é um jogo eletrônico de futebol no estilo fantasy game, um jogo fictício no qual as pessoas montam seus times com jogadores de futebol da vida real, lançado no ano de 2004. Foi criado e é mantido pelo site Globo.com e promovido pelo canal de TV por assinatura SporTV e também pela Globo.

<hr>

## Definição do Problema

### Previsão de Escalação

Realizar previsão da pontuação dos jogadores do Cartola FC.

### Dados do problema
- Informações importantes quanto a pontuação:
    - Scouts Positivos:
        - DS - Desarme (+ 1,0)
        - G - Gol (+8,0)
        - A - Assistência (+5,0)
        - SG - Jogos sem Sofrer Gols (+5,0)
        - FS - Falta Sofrida ( +0,5)
        - FF - Finalização para Fora (+0,8)
        - FD - Finalização Defendida (+1,2)
        - FT - Finalização na Trave (+3,0)
        - DD - Defesa Difícil (+4,0)
        - DP - Defesa de Pênalti (+7,0)
    - Scouts Negativos:
        - GC - Gol Contra (-5,0)
        - CV - Cartão Vermelho (-5,0)
        - CA - Cartão Amarelo (-2,0)
        - GS - Gol Sofrido (-2,0)
        - PP - Pênalti Perdido (-4,0)
        - FC - Falta Cometida (-0,5)
        - I - Impedimento (-0,5)
        - PI - Passe Incompleto (-0,1)



<hr>

## Desenvolvimento do Projeto



In [92]:
# Imports do projeto
# analise de dados
import pandas as pd
import numpy as np
import random as rnd

# Requests e Elementos HTML
import re                          # Expressão regulares
import requests                    # Acessar páginas da internet
from bs4 import BeautifulSoup      # Raspar elementos de páginas da internet

# visualização
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate

# metricas 
from sklearn.metrics import classification_report

## Captura dos dados 

Os dados são baixados diretamente do diretório do do GitHub: https://github.com/henriquepgomide/caRtola/tree/master/data/2020

In [2]:

# URL com caminho do repositório
URL = 'https://github.com/henriquepgomide/caRtola/tree/master/data/2020'
html = requests.get(URL)
# Criar objeto BeautifulSoup para raspar urls 
soup = BeautifulSoup(html.text, 'lxml')

In [3]:
# Formatar HTML para extrair os dados
result = []
for tag in soup.find_all('a', href=True):
    result.append(tag)
result[70:90:2]

[<a class="f6 link-gray text-mono ml-2 d-none d-lg-inline" data-pjax="" href="/henriquepgomide/caRtola/commit/fe70dcf565ce007105f4eccd386be7d0b7c43f96">fe70dcf</a>,
 <a class="link-gray-dark text-bold" data-pjax="true" href="/henriquepgomide/caRtola/commit/fe70dcf565ce007105f4eccd386be7d0b7c43f96">fix round 10 file</a>,
 <a class="d-none js-permalink-shortcut" data-hotkey="y" href="/henriquepgomide/caRtola/tree/fe70dcf565ce007105f4eccd386be7d0b7c43f96/data/2020">Permalink</a>,
 <a class="js-navigation-open link-gray-dark" href="/henriquepgomide/caRtola/tree/master/data/2020/team-features" id="015cc237790ac744a369fe0a639f3b4b-40cbb9d10b172966a10d0cc0beb858725ca89218" title="team-features">team-features</a>,
 <a class="js-navigation-open link-gray-dark" href="/henriquepgomide/caRtola/blob/master/data/2020/2020-medias-jogadores.csv" id="afd3e0c547e8d3b59b84300bfc9c9a1b-02e0d797e1be0e45f5e0b9c0357c3626e238604c" title="2020-medias-jogadores.csv">2020-medias-jogadores.csv</a>,
 <a class="js-

In [64]:
regex =  '\.csv$'

dict_of_files = {}                                                          # Criar dicionário vazio
for tag in soup.find_all('a', attrs={'href': re.compile(regex)}):           # Encontrar tags de nosso interesse
    href_str = tag.get('href')                                              
    file_name = re.sub('/henriquepgomide/caRtola/blob/master/data/2020/',   # Substituir padrão por nada
                       '', 
                       href_str)
    
    file_url = re.sub('/henriquepgomide/caRtola/blob/master/data/2020/',    # Substituir padrão por links para arquivos raw
                    'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/', 
                    href_str)

    dict_of_files[file_name] = file_url

In [65]:
#Lista dos files
dict(list(dict_of_files.items()))

{'2020-medias-jogadores.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/2020-medias-jogadores.csv',
 '2020_partidas.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/2020_partidas.csv',
 'rodada-1.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-1.csv',
 'rodada-10.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-10.csv',
 'rodada-11.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-11.csv',
 'rodada-2.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-2.csv',
 'rodada-3.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-3.csv',
 'rodada-4.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-4.csv',
 'rodada-5.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rod

In [66]:
# Ler os dataframes das rodadas
list_of_dataframes = []
for key, item in list(dict_of_files.items())[2:14]:
    df = pd.read_csv(item)
    df['rodada'] = key
    list_of_dataframes.append(df)

In [67]:
# Combinar lista de DataFrames criados
df_jogadores = pd.concat(list_of_dataframes)



In [68]:
# Ler os dataframes das partidas 
for key, item in list(dict_of_files.items())[1:2]:
    df_jogos = pd.read_csv(item)


<hr>

## Análise descritiva
A análise de dados descritiva apresenta a separação e os tipos das características bem como quais colunas possuem valores vazios. Além de apresentar uma visão resumida da distribuição do dataset.

#### Insights encontradas na análise: 
        
        - Muitas colunas com dados faltantes
        - Maior média de pontos encontrada na característica `DS` (desarme).
        - Existem mais registros dos jogadores de Meio-Campo.

#### Features categóricas x numéricas
- categóricas: `atletas.status_id`, `atletas.nome`, `atletas.slug` , `atletas.apelido`, `atletas.foto`, `tletas.clube.id.full.name`, `rodada`, `atletas.posicao_id`

- numéricas: `atletas.rodada_id	`, `atletas.clube_id`, `atletas.pontos_num`, `atletas.preco_num`, `atletas.variacao_num`, `atletas.media_num`, `atletas.jogos_num`, `FF`, `FS`, `G`, `PI`, `CA`,`FC`, `DS`, `FT`, `DD`, `GS` , `FD`, `GC`, `SG`, `A`, `I`, `CV`,	`PP`, `DP`

In [72]:
# Colunas do dataset
df_jogadores.columns

Index(['Unnamed: 0', 'atletas.nome', 'atletas.slug', 'atletas.apelido',
       'atletas.foto', 'atletas.atleta_id', 'atletas.rodada_id',
       'atletas.clube_id', 'atletas.posicao_id', 'atletas.status_id',
       'atletas.pontos_num', 'atletas.preco_num', 'atletas.variacao_num',
       'atletas.media_num', 'atletas.jogos_num', 'atletas.clube.id.full.name',
       'FF', 'FS', 'G', 'PI', 'CA', 'FC', 'DS', 'FT', 'DD', 'GS', 'FD', 'GC',
       'SG', 'A', 'I', 'CV', 'PP', 'rodada', 'DP'],
      dtype='object')

In [73]:
#Preview das informações contidas no dataset
df_jogadores.head(4)

,Unnamed: 0,atletas.nome,atletas.slug,atletas.apelido,atletas.foto,atletas.atleta_id,atletas.rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.jogos_num,atletas.clube.id.full.name,FF,FS,G,PI,CA,FC,DS,FT,DD,GS,FD,GC,SG,A,I,CV,PP,rodada,DP
0,1,Jorge de Moura Xavier,jorginho,Jorginho,https://s.glbimg.com/es/sde/f/2020/02/27/85109...,77544,1,373,mei,Provável,0.0,6.0,0.0,0.0,0,Atlético-GO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-1.csv,NaN
1,2,Vagner Carmo Mancini,vagner-mancini,Vagner Mancini,https://s.glbimg.com/es/sde/f/2020/07/16/1dd17...,39850,1,373,tec,Provável,0.0,2.0,0.0,0.0,0,Atlético-GO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-1.csv,NaN
2,3,Moacir Costa da Silva,moacir,Moacir,https://s.glbimg.com/es/sde/f/2020/02/27/9937a...,60858,1,373,mei,Nulo,0.0,3.0,0.0,0.0,0,Atlético-GO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-1.csv,NaN
3,4,Leonardo Cittadini,leo-cittadini,Léo Cittadini,https://s.glbimg.com/es/sde/f/2019/03/30/47516...,70986,1,293,mei,Provável,9.8,12.9,4.9,9.8,1,Athlético-PR,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-1.csv,NaN


In [75]:
# Tamanho das colunas
df_jogadores.shape

(7854, 35)

#### Total de features por Tipo dos dados
- Int64: 5
- Object: 8 
- Float64: 22

#### Dados faltantes
Verificado que em 18 colunas do dataset existem dados faltantes (missing)

In [11]:
df_jogadores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7854 entries, 0 to 711
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  7854 non-null   int64  
 1   atletas.nome                7854 non-null   object 
 2   atletas.slug                7854 non-null   object 
 3   atletas.apelido             7854 non-null   object 
 4   atletas.foto                7792 non-null   object 
 5   atletas.atleta_id           7854 non-null   int64  
 6   atletas.rodada_id           7854 non-null   int64  
 7   atletas.clube_id            7854 non-null   int64  
 8   atletas.posicao_id          7854 non-null   object 
 9   atletas.status_id           7854 non-null   object 
 10  atletas.pontos_num          7854 non-null   float64
 11  atletas.preco_num           7854 non-null   float64
 12  atletas.variacao_num        7854 non-null   float64
 13  atletas.media_num           7854 n

In [76]:
# Verificar dados NaN
df_jogadores.isna().sum()

Unnamed: 0                       0
atletas.nome                     0
atletas.slug                     0
atletas.apelido                  0
atletas.foto                    62
atletas.atleta_id                0
atletas.rodada_id                0
atletas.clube_id                 0
atletas.posicao_id               0
atletas.status_id                0
atletas.pontos_num               0
atletas.preco_num                0
atletas.variacao_num             0
atletas.media_num                0
atletas.jogos_num                0
atletas.clube.id.full.name       0
FF                            5365
FS                            4318
G                             6944
PI                            3382
CA                            5968
FC                            4103
DS                            4114
FT                            7489
DD                            7592
GS                            7580
FD                            5938
GC                            7818
SG                  

### Distribuição dos dados
Verificação da distribuição das colunas numéricas e categóricas

In [80]:
# Distribuição numérica
df_jogadores.describe()

,Unnamed: 0,atletas.atleta_id,atletas.rodada_id,atletas.clube_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.jogos_num,FF,FS,G,PI,CA,FC,DS,FT,DD,GS,FD,GC,SG,A,I,CV,PP,DP
count,7854.000000,7854.000000,7854.000000,7854.000000,7854.000000,7854.000000,7854.000000,7854.000000,7854.000000,2489.000000,3536.000000,910.000000,4472.000000,1886.000000,3751.000000,3740.00000,365.000000,262.000000,274.000000,1916.000000,36.0,1104.000000,741.000000,1018.000000,164.000000,45.0,18.0
mean,357.507257,87198.332824,5.998472,289.613318,1.047255,4.767907,0.001213,1.408136,2.441558,2.693451,5.057975,2.385714,19.034884,1.507953,5.096241,6.52246,1.101370,4.362595,4.890511,2.004697,1.0,1.602355,1.145749,1.760314,1.018293,1.0,1.0
std,206.139550,18891.625484,3.163646,31.142449,2.778230,3.514188,0.861508,2.254622,2.734732,2.197549,5.098773,6.580099,18.971630,1.470376,4.229114,6.13142,0.311189,3.398531,3.350532,1.585666,0.0,0.960901,0.403128,1.244661,0.134418,0.0,0.0
min,1.000000,36943.000000,1.000000,262.000000,-8.400000,0.720000,-6.610000,-7.200000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0
25%,179.000000,78077.000000,3.000000,267.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,5.000000,1.000000,2.000000,2.00000,1.000000,2.000000,2.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0
50%,357.500000,91736.000000,6.000000,282.000000,0.000000,4.000000,0.000000,0.300000,1.000000,2.000000,3.000000,1.000000,13.000000,1.000000,4.000000,5.00000,1.000000,3.500000,4.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0
75%,536.000000,102292.000000,9.000000,293.000000,0.900000,6.480000,0.000000,2.470000,4.000000,4.000000,7.000000,2.000000,26.250000,2.000000,7.000000,9.00000,1.000000,6.000000,7.000000,2.000000,1.0,2.000000,1.000000,2.000000,1.000000,1.0,1.0
max,720.000000,109963.000000,11.000000,373.000000,26.600000,25.410000,10.210000,19.300000,11.000000,17.000000,55.000000,113.000000,129.000000,34.000000,32.000000,43.00000,3.000000,17.000000,16.000000,11.000000,1.0,6.000000,3.000000,9.000000,2.000000,1.0,1.0


In [79]:
# Distribuição das variáveis categóricas
df.describe(include=['O'])

,atletas.nome,atletas.slug,atletas.apelido,atletas.foto,atletas.posicao_id,atletas.status_id,atletas.clube.id.full.name,rodada
count,712,712,712,707,712,712,712,712
unique,712,649,653,707,6,5,20,1
top,Matheus Nascimento de Paula,luiz-henrique,Gabriel,https://s.glbimg.com/es/sde/f/2020/07/21/282e8...,mei,Nulo,Internacional,rodada-9.csv
freq,1,4,4,1,219,381,42,712


## Análise por cruzamento e agrupamento de dados

In [94]:
# Registros por posição de jogador
df_jogadores['atletas.posicao_id'].value_counts()

mei    2483
ata    1862
zag    1265
lat    1210
gol     819
tec     215
Name: atletas.posicao_id, dtype: int64

In [93]:
# registros por clubes
df_jogadores['atletas.clube_id'].value_counts()

290    459
265    458
267    449
285    443
293    433
294    417
284    396
292    394
263    383
354    383
373    382
280    382
266    381
282    373
276    369
264    363
356    359
262    359
277    337
275    334
Name: atletas.clube_id, dtype: int64

In [121]:
# Registros por status de jogador
df_jogadores.groupby('atletas.status_id')['atletas.atleta_id'].count()

atletas.status_id
Contundido     385
Dúvida         358
Nulo          4726
Provável      2302
Suspenso        83
Name: atletas.atleta_id, dtype: int64

In [89]:
#Total de registos por estação
df_jogadores.groupby(['atletas.posicao_id', 'atletas.pontos_num'])['atletas.posicao_id'].count()

atletas.posicao_id  atletas.pontos_num
ata                 -6.0                  1
                    -4.8                  2
                    -4.2                  1
                    -4.1                  2
                    -3.8                  1
                                         ..
zag                  12.4                 1
                     12.7                 1
                     12.8                 1
                     16.1                 1
                     16.2                 1
Name: atletas.posicao_id, Length: 900, dtype: int64

<hr>

### Primeiro ajuste do dataset para unificação/input com informação do dataset df_jogos

Antes de ser realizada a análise por visulização de dados foi unificado o dataset de jogos no data set df_jogadores, e para tal, foi foi necessário fazer alguns ajustes com os dados faltantes dos dois dataframes.
- Dropado colunas `atletas.foto`, `rodada`,  `atletas.slug`, `atletas.nome`, `atletas.clube.id.full.name`
- Ajudado labels das colunas relacionados com a rodada para rodada_id para ambos dataFrames

In [122]:
# Remover colunas atletas.foto e rodada
df_jogadores.drop(['atletas.foto', 'rodada', 'atletas.slug', 'atletas.nome', 'atletas.clube.id.full.name'], axis=1, inplace=True)

In [123]:
# Remover linhas cuja coluna rodada_id esteja NaN
df_jogadores = df_jogadores[df_jogadores['atletas.rodada_id'].notnull()]

# Remover coluna Unnamed
df_jogadores = df_jogadores.loc[:, ~df_jogadores.columns.str.contains('^Unnamed')]

In [125]:
# Remover jogadores contundios e suspensos da rodada
check_colunas_null = ['A','CA','CV','DD','DP','DS','FC','FD','FF','FS','FT','G','GC','GS','I','PI','PP','SG']
df_jogadores = df_jogadores.dropna(subset=check_colunas_null, how='all')

In [126]:
 # preencher com 0 colunas NaN
 df_jogadores.fillna(0, inplace=True)


In [127]:
df_jogadores.shape

(4704, 29)

### Validação dos dados do dataframe jogos

In [22]:
# Verificar dados NaN no dataset das Partidas
df_jogos.isna().sum()

date           0
home_team      0
away_team      0
home_score    16
away_score    16
round          0
dtype: int64

In [132]:
# Remover linhas NaN pelo home score
df_jogos.dropna(subset=['home_score'], inplace=True)

In [133]:
# Renomear colunas que possuem a número da rodada para rodada_id em ambos dataset
df_jogos.rename(columns={'round':'rodada_id'}, inplace=True)
df_jogadores.rename(columns={'atletas.rodada_id':'rodada_id'}, inplace=True)

In [25]:
df_jogadores.tail(3)

,atletas.apelido,atletas.atleta_id,rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,...,DD,GS,FD,GC,SG,A,I,CV,PP,DP
706,Rentería,95356,9,263,mei,Nulo,0.6,5.62,-1.38,0.60,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
709,Cayo Tenório,107367,9,267,lat,Nulo,0.0,4.88,0.00,4.33,...,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
711,Luiz Henrique,109282,9,266,ata,Nulo,0.2,1.50,-0.48,1.01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Input/Unificação dos dados no dataframe dos jogadores
- Adicionado colunas `Adversario`, `jogoForaOuCasa`, `golFavor`, `golFora`  

In [134]:
# Procura qual adversário por rodade e atleta
def getAdversario(clube_id, rodada_id):
    advFora = df_jogos.loc[(df_jogos['home_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['away_team']
    advCasa = df_jogos.loc[(df_jogos['away_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['home_team']
    if (advFora is not None) and (advFora.values > 0):
        return int(advFora.values[0])
    else :
         if (advCasa is not None) and (advCasa.values > 0):
               return int(advCasa.values[0])


In [135]:
# procura se o jogo foi em casa ou f
def getJogoForaEmCasa(clube_id, rodada_id):
    jogouEmCasa = df_jogos.loc[(df_jogos['home_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['home_team']
    jogouFora = df_jogos.loc[(df_jogos['away_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['away_team']
    if (jogouEmCasa is not None) and (jogouEmCasa.values > 0):
        return 0
    else :
         if (jogouFora is not None) and (jogouFora.values > 0):
               return 1


In [136]:
def gerarColunasGols(linha_df):
    jogoRealizadoEmCasa = df_jogos.loc[(df_jogos['home_team'] == linha_df['atletas.clube_id']) & (df_jogos['rodada_id'] == linha_df['rodada_id'])]
    jogoRealizadoFora = df_jogos.loc[(df_jogos['away_team'] == linha_df['atletas.clube_id']) & (df_jogos['rodada_id'] == linha_df['rodada_id'])]
    if not jogoRealizadoEmCasa.empty:
        linha_df['golFavor'] = jogoRealizadoEmCasa['home_score'].values[0]
        linha_df['golContra'] = jogoRealizadoEmCasa['away_score'].values[0]
    else:
        if not jogoRealizadoFora.empty:
            linha_df['golFavor'] = jogoRealizadoFora['away_score'].values[0]
            linha_df['golContra'] = jogoRealizadoFora['home_score'].values[0]
    return linha_df
        

In [137]:
# Gera novas colunas com os gols do time a favor ou contra para cada rodada
df_jogadores = df_jogadores.apply(lambda x: gerarColunasGols(x), axis=1)

In [138]:
# Adversário do time/jogador
df_jogadores['adversario'] = df_jogadores.apply(lambda x: getAdversario(x['atletas.clube_id'], x['rodada_id']), axis=1)

In [139]:
# Jogo fora ou em casa
df_jogadores['jogoForaOuCasa'] = df_jogadores.apply(lambda x: getJogoForaEmCasa(x['atletas.clube_id'], x['rodada_id']), axis=1)

In [140]:
# remover linhas cuja coluna rodada_id esteja NaN
df_jogadores.dropna(subset=['adversario'], inplace=True)

In [141]:
pd.set_option('display.max_columns', None)
df_jogadores.tail(30)

,A,CA,CV,DD,DP,DS,FC,FD,FF,FS,FT,G,GC,GS,I,PI,PP,SG,atletas.apelido,atletas.atleta_id,atletas.clube_id,atletas.jogos_num,atletas.media_num,atletas.pontos_num,atletas.posicao_id,atletas.preco_num,atletas.status_id,atletas.variacao_num,golContra,golFavor,rodada_id,adversario,jogoForaOuCasa
643,1.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,9.0,0.0,0.0,Ygor Catatau,100977,267,3,1.70,-0.1,ata,1.95,Dúvida,-0.12,2.0,1.0,9,373.0,0.0
645,0.0,1.0,0.0,0.0,0.0,15.0,6.0,1.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,35.0,0.0,2.0,Jonathan,86713,294,7,2.93,3.6,lat,4.47,Provável,0.40,3.0,3.0,9,290.0,1.0
646,0.0,1.0,0.0,0.0,0.0,6.0,4.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,Ratinho,101841,290,3,1.17,0.0,mei,2.10,Nulo,0.00,3.0,3.0,9,294.0,0.0
647,0.0,0.0,0.0,0.0,0.0,16.0,5.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,63.0,0.0,1.0,Mariano,38877,282,5,2.98,2.5,lat,8.85,Nulo,-1.16,3.0,1.0,9,277.0,1.0
648,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,Ruan,104153,264,4,0.77,0.0,mei,1.22,Contundido,0.00,2.0,0.0,9,275.0,0.0
649,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,David Duarte,91772,290,2,3.40,4.2,zag,3.42,Provável,0.51,3.0,3.0,9,294.0,0.0
650,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Edson,105901,265,3,2.10,2.3,mei,1.62,Dúvida,0.15,2.0,0.0,9,284.0,0.0
654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,Bruno Leite,102301,293,1,-0.60,0.0,mei,0.77,Nulo,0.00,1.0,1.0,9,263.0,0.0
655,0.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Daniel Oliveira,106076,290,2,2.75,0.0,lat,2.15,Nulo,0.00,3.0,3.0,9,294.0,0.0
658,0.0,1.0,0.0,0.0,0.0,5.0,3.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,Lucas Evangelista,84674,280,5,0.86,1.4,mei,3.06,Nulo,0.26,1.0,1.0,9,276.0,1.0


<hr>

### Análise por visualização dos dados

### Correção e limpeza final dos dados
    - Remover treinadores do dataset
    - Remover atletas com pountuação zerada e contundido

In [145]:
# Remover técnicos do dataset
df_jogadores = df_jogadores[df_jogadores['atletas.posicao_id'] != 'tec']

In [146]:
# Remover técnicos do dataset
df_jogadores = df_jogadores[ (df_jogadores['atletas.pontos_num'] != 0) & (df_jogadores['atletas.status_id'] != 'Contundido')]

In [147]:
df_jogadores[(df_jogadores['atletas.clube_id'] == 262) & (df_jogadores['atletas.atleta_id'] == 38509)]

,A,CA,CV,DD,DP,DS,FC,FD,FF,FS,FT,G,GC,GS,I,PI,PP,SG,atletas.apelido,atletas.atleta_id,atletas.clube_id,atletas.jogos_num,atletas.media_num,atletas.pontos_num,atletas.posicao_id,atletas.preco_num,atletas.status_id,atletas.variacao_num,golContra,golFavor,rodada_id,adversario,jogoForaOuCasa
11,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,9.0,0.0,0.0,Diego Alves,38509,262,1,5.60,5.6,gol,14.21,Provável,0.21,1.0,0.0,1,282.0,0.0
110,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,9.0,0.0,0.0,Diego Alves,38509,262,2,-0.45,-6.5,gol,9.28,Suspenso,-4.93,3.0,0.0,2,373.0,1.0
128,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0,0.0,13.0,0.0,0.0,Diego Alves,38509,262,3,-1.10,-2.4,gol,7.33,Provável,-1.95,1.0,1.0,4,284.0,0.0
28,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,14.0,0.0,0.0,Diego Alves,38509,262,4,-1.35,-2.1,gol,6.58,Provável,-0.75,1.0,1.0,5,263.0,0.0
681,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,19.0,0.0,1.0,Diego Alves,38509,262,5,2.22,16.5,gol,10.72,Dúvida,4.14,0.0,1.0,6,277.0,1.0


<hr>

### Ajuste do DataFrame de Teste

In [197]:
#total geral de unidades por linha
df_teste['total_un'] = df_teste.apply(lambda x: (x['Original_473'] + x['Original_269'] + x['Zero'] + x['Maçã-Verde'] + x['Tangerina'] + x['Citrus'] + x['Açaí-Guaraná'] + x['Pêssego'] ), axis=1)
df_teste = df_teste.drop(['Tempo', 'Estação', 'Movimentação'],  axis=1)
df_teste.head(50)


,LAT,LONG,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,total_un
0,-23.5489,-46.6388,22,65,22,14,3,26,43,18,213
1,-23.5489,-46.6388,11,56,16,14,43,20,37,15,212
2,-23.5489,-46.6388,86,51,8,11,41,20,35,12,264
3,-23.5489,-46.6388,76,47,65,11,37,14,32,10,292
4,-23.5489,-46.6388,68,42,60,6,35,13,26,5,255
5,-23.5489,-46.6388,57,35,55,43,31,9,22,43,295
6,-23.5489,-46.6388,45,33,50,39,25,5,16,39,252
7,-23.5489,-46.6388,34,28,41,39,22,43,13,34,254
8,-23.5489,-46.6388,22,24,31,35,18,40,8,29,207
9,-23.5489,-46.6388,9,16,26,33,13,38,4,24,163


In [198]:
X_train = df.drop("TARGET", axis=1)
Y_train = df["TARGET"]
X_test = df_teste

In [200]:
#Balanceamento das classes com menos registros no dataset treino
smt = SMOTE()
X_train, Y_train = smt.fit_sample(X_train,Y_train)

In [201]:
#Normalização dos dados
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


### Treinando um classificador com base em no modelo Random Forest

In [212]:
#Algorítomo Machine Learning 
random_forest = RandomForestClassifier(n_estimators=80, max_depth=11, max_features=5, bootstrap=True, min_samples_leaf=3, min_samples_split=7)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)

#importantes = pd.DataFrame(random_forest.feature_importances_, index= X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
#importantes
# Acurácia alcançada pela árvore de decisão
#print("Acurácia: {}%".format(100*round(accuracy_score(y_test, Y_pred), 2)))

In [213]:
# Acurácia alcançada pela árvore de decisão
round(random_forest.score(X_train, Y_train) * 100, 2)

96.04

### Submissão

In [210]:
result = df_teste.assign(TARGET=Y_pred)
result.head(50)

,LAT,LONG,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,total_un,TARGET
0,-23.5489,-46.6388,22,65,22,14,3,26,43,18,213,REABASTECER
1,-23.5489,-46.6388,11,56,16,14,43,20,37,15,212,REABASTECER
2,-23.5489,-46.6388,86,51,8,11,41,20,35,12,264,REABASTECER
3,-23.5489,-46.6388,76,47,65,11,37,14,32,10,292,NORMAL
4,-23.5489,-46.6388,68,42,60,6,35,13,26,5,255,REABASTECER
5,-23.5489,-46.6388,57,35,55,43,31,9,22,43,295,NORMAL
6,-23.5489,-46.6388,45,33,50,39,25,5,16,39,252,REABASTECER
7,-23.5489,-46.6388,34,28,41,39,22,43,13,34,254,NORMAL
8,-23.5489,-46.6388,22,24,31,35,18,40,8,29,207,NORMAL
9,-23.5489,-46.6388,9,16,26,33,13,38,4,24,163,REABASTECER


In [211]:
#Exportar submissão
result.to_csv('results.csv', index=False)